In [6]:
import tiktoken

enc = tiktoken.get_encoding("gpt2")

In [19]:

def add_prefix(example):
    example['text'] = "1"
    out = {"ids": [1]}
    return out

for i in dataset.map(add_prefix, remove_columns="text"):
    print(i)
    break

{'ids': [1]}


In [ ]:
for i in dataset.shard(num_shards=10, index=1)

IterableDataset({
    features: ['text'],
    num_shards: 2
})

In [1]:
import datamodule
from importlib import reload
reload(datamodule)
dm = datamodule.TokenDataModule("config.json")
dm.prepare_data()
dm.setup()

dl = dm.train_dataloader()

for i in dl:
    print(i)
    break

/home/brimax/miniconda3/envs/brimax/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[tensor([[18988,  5642,    11,  ...,    13,   198,   198],
        [  262,  4097, 44634,  ...,   866,  3688,   284],
        [   83,   787,   683,  ...,   247,    82, 22830],
        ...,
        [ 1454,  3019,   992,  ..., 41414,   559,  6717],
        [ 5913,  2282,  3292,  ...,   290,   645, 22830],
        [14649,  1377, 40338,  ...,  2634,   531,    13]], dtype=torch.uint16), tensor([[False, False,  True,  ..., False,  True, False],
        [False, False,  True,  ..., False, False, False],
        [False, False, False,  ..., False,  True, False],
        ...,
        [False, False, False,  ..., False,  True, False],
        [False, False, False,  ..., False, False, False],
        [False,  True,  True,  ..., False, False,  True]])]


In [4]:
i[0].dtype

torch.uint16

In [24]:
0.15*512

76.8

In [31]:
i[1].sum(axis= 1).sum() / i[1].shape[0]

tensor(78.5938)

In [ ]:
import numpy as np
m = np.memmap('./data/openwebtext_10k/openwebtext.bin', dtype=np.uint16, mode='r')

In [31]:
enc.decode(m)

'Port-au-Prince, Haiti (CNN) -- Earthquake victims, writhing in pain and grasping at life, watched doctors and nurses walk away from a field hospital Friday night after a Belgian medical team evacuated the area, saying it was concerned about security.\n\nThe decision left CNN Chief Medical Correspondent Sanjay Gupta as the only doctor at the hospital to get the patients through the night.\n\nCNN initially reported, based on conversations with some of the doctors, that the United Nations ordered the Belgian First Aid and Support Team to evacuate. However, Belgian Chief Coordinator Geert Gijs, a doctor who was at the hospital with 60 Belgian medical personnel, said it was his decision to pull the team out for the night. Gijs said he requested U.N. security personnel to staff the hospital overnight, but was told that peacekeepers would only be able to evacuate the team.\n\nHe said it was a "tough decision" but that he accepted the U.N. offer to evacuate after a Canadian medical team, also

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/brimax/miniconda3/envs/brimax/lib/python3.12/site-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/brimax/miniconda3/envs/brimax/lib/python3.12/site-packages/torch/utils/data/_utils/fetch.py", line 50, in fetch
    data = self.dataset.__getitems__(possibly_batched_index)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/brimax/miniconda3/envs/brimax/lib/python3.12/site-packages/torch/utils/data/dataset.py", line 420, in __getitems__
    return [self.dataset[self.indices[idx]] for idx in indices]
            ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^
  File "/home/brimax/repos/diffusion-llms/diffusion-llms/datamodule.py", line 36, in __getitem__
    mask = torch.rand(size=(sequence.shape[0]), dtype=torch.float32) < 0.15
   

In [41]:
PATH = "./data/openwebtext_10k.txt"
with open(PATH, "w") as f:
    for i in dataset["train"]:
        f.write(i["text"] + "\n")

In [ ]:
import tiktoken

enc = tiktoken.get_encoding("gpt2")
encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
decode = lambda l: enc.decode(l)

In [12]:
encode("hello world, how are you? fadsf")

[31373, 995, 11, 703, 389, 345, 30, 277, 5643, 69]

In [15]:
import ast

ast.literal_eval("hello")

ValueError: malformed node or string on line 1: <ast.Name object at 0x7f19953dca10>

In [ ]:
import json
import wandb
import torch
from model import GPT2
import tiktoken

CONFIG_PATH = "./config.json"

with open(CONFIG_PATH, "r") as f:
    config = json.load(f)

model = GPT2(CONFIG_PATH)

print("Successfully loaded GPT-2")

# Test model Logic
# 1 sentence of len 10, from a vocab of 100 toks
T = 10
sentence = torch.randint(0, 100, size=(1, T+1))
X = sentence[:, :-1]
y = sentence[:, 1:]

out = model.step(
    batch = (X, y),     # read in as idx, targets = batch
    batch_idx = 0  
)

# The loss
print(out)

model.generate(X, max_new_tokens=512)

number of parameters: 7.23M
Successfully loaded GPT-2
tensor(10.8942, grad_fn=<NllLossBackward0>)


tensor([[   49,    22,    19,    61,    95,    39,     3,    39,    96,    26,
          9123, 47914, 39695, 46040, 28352,  7122,   310,   375, 12712,  1945,
         13487, 30950,  1259, 41498, 41162, 28550, 34179, 36172, 25174, 10526,
         47161, 20574, 21792, 46750,  1110,  8111, 49940, 40032, 36511, 10322,
         40220, 38804,  6291, 15472,  6383, 38105, 39885, 45026, 37103,  2993,
         32006, 49261,  2810, 15778, 32221, 25724, 38034, 33058,   567, 32690,
         32740, 19129, 39442, 21769,  5761, 46829,  3810, 49484,  2618, 23049,
         39007, 13152,  9365, 22517, 48877, 47661, 48294, 18434, 50176, 45629,
         29568,  1462, 20523, 41217, 33363, 19523, 11484,   479,  4547,  8619,
          4852,  1432, 47325, 39527,  4116, 31367, 34746, 13218, 45626,  3284,
         41936, 21447, 49821, 10265, 22147,   716,  1351, 26055,  2650, 22228,
         34509, 39176, 44710, 44167, 26699, 20248, 45573, 41015, 38608, 34034,
         22475,  4021, 35074, 38361, 17299, 22627, 3

In [ ]:
model.generate()

In [8]:
help(model.gpt2)

Help on GPT2Model in module transformers.models.gpt2.modeling_gpt2 object:

class GPT2Model(GPT2PreTrainedModel)
 |  GPT2Model(config)
 |
 |  The bare GPT2 Model transformer outputting raw hidden-states without any specific head on top.
 |
 |  This model inherits from [`PreTrainedModel`]. Check the superclass documentation for the generic methods the
 |  library implements for all its model (such as downloading or saving, resizing the input embeddings, pruning heads
 |  etc.)
 |
 |  This model is also a PyTorch [torch.nn.Module](https://pytorch.org/docs/stable/nn.html#torch.nn.Module) subclass.
 |  Use it as a regular PyTorch Module and refer to the PyTorch documentation for all matter related to general usage
 |  and behavior.
 |
 |  Parameters:
 |      config ([`GPT2Config`]): Model configuration class with all the parameters of the model.
 |          Initializing with a config file does not load the weights associated with the model, only the
 |          configuration. Check out the